In [4]:
import jieba
import codecs as cs
import pickle
#import time
import numpy as np
from keras_bert import Tokenizer,get_custom_objects
from keras.models import load_model

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [2]:
test_corpus = pickle.load(open(r'C:\Users\HashiriNio\Desktop\final_design\Final_one\data\corpus_test.pkl','rb'))
test_questions = [test_corpus[i]['question'] for i in range(len(test_corpus))]
test_entitys = [test_corpus[i]['gold_entitys'] for i in range(len(test_corpus))]
test_entitys = [[entity[1:-1].split('_')[0] for entity in line]for line in test_entitys]

In [2]:
class EntityLinker():
    def __init__(self):
        self.seq_max_len = 20
        #加载bert字典
        dict_path = r'C:\Users\HashiriNio\Desktop\final_design\Final_one\bert\vocab.txt'
        token_dict = {}
        with cs.open(dict_path, 'r', 'utf8') as reader:
            for line in reader:
                token = line.strip()
                token_dict[token] = len(token_dict)
        self.tokenizer = Tokenizer(token_dict)
        
        #加载NER模型
        model_path = r'C:\Users\HashiriNio\Desktop\final_design\Final_one\data\model\model_ner_general.h5'
        custom_objects = get_custom_objects()
        self.ner_model = load_model(model_path,custom_objects =custom_objects)
        
        #加载实体词典
        mention_dic_path = r'C:\Users\HashiriNio\Desktop\final_design\Final_one\data\mention_dict.txt'
        mention_dict = {}
        with open(mention_dic_path,'r',encoding = 'utf-8') as f:
            for i in f:
                if i.strip():
                    mention_dict[i.strip()] = 1
        self.mention_dict = mention_dict
        #jieba.load_userdict('mention_dic_path')
        print('clear')
    
    def restore_entity_from_corpus(self,predicty,question):
        def restore_entity_from_labels(labels,question):
            entitys = []
            str = ''
            labels = labels[1:-1]
            #print(labels,question)
            for i in range(min(len(labels),len(question))):
                if labels[i]==1:
                    str += question[i]
                else:
                    if len(str):
                        entitys.append(str)
                        str = ''
            if len(str):
                entitys.append(str) 
            return entitys
        all_entitys = []
        for i in range(len(predicty)):
            all_entitys.append(restore_entity_from_labels(predicty[i],question[i]))
            #print(predicty[i])
        return all_entitys
    
    def mention_get(self,question):
        #ner
        q = question
        mention_ner = []
        i1,i2 = self.tokenizer.encode(first = q,max_len = self.seq_max_len)
        i1 = np.array(i1)
        i2 = np.array(i2)        
        pre = self.ner_model.predict([np.array([i1,i2]),np.array([i2,i2])])
        pre = [1 if i >0.5 else 0 for i in pre[0]]
        #print(pre)
        predicty = self.restore_entity_from_corpus([pre],[q])
        print(predicty)
        mention_ner = predicty[0]
        #for i in self.mention_dict.keys():
        #    for j in predicty[0]:
        #       if j in i:
        #            mention_ner.append(i)
        
        #分词
        mentions = []
        tokens = jieba.lcut(question)
        for t in tokens:
            if t in self.mention_dict.keys():
                mentions.append(t)
        
        mentionall = mentions + mention_ner
        return mentionall

In [3]:
question = '计算机网络是什么'
tokens = jieba.lcut(question)
print(tokens)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HASHIR~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.673 seconds.
Prefix dict has been built successfully.


['计算机网络', '是', '什么']


In [3]:
me = EntityLinker()

clear


In [4]:
q = '伯瓦尔·弗塔根在游戏《魔兽世界》中的身份是？'
entity = me.mention_get(q)
entity

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HASHIR~1\AppData\Local\Temp\jieba.cache


[['伯瓦尔·弗塔根', '魔兽世界']]


Loading model cost 0.738 seconds.
Prefix dict has been built successfully.


['伯',
 '瓦尔',
 '·',
 '在',
 '游戏',
 '《',
 '魔兽',
 '世界',
 '》',
 '中',
 '的',
 '身份',
 '是',
 '？',
 '伯瓦尔·弗塔根',
 '魔兽世界']

In [14]:
question = '伯瓦尔·弗塔根在游戏《魔兽世界》中的身份是？'
#tokens = jieba.lcut(question)
tokens = ['伯', '瓦尔', '·', '在', '游戏', '《', '魔兽', '世界', '》', '中', '的', '身份', '是', '？','傻逼']
mentions=[]
for t in tokens:
    for m in me.mention_dict.keys():
        if m == t:
            mentions.append(t)
mentions

['伯',
 '瓦尔',
 '·',
 '在',
 '游戏',
 '《',
 '魔兽',
 '世界',
 '》',
 '中',
 '的',
 '身份',
 '是',
 '？',
 '傻逼']

In [12]:
list(me.mention_dict.keys())[111500:111510]

['锁清秋',
 '张志华',
 '熊彦',
 '手绘效果图表现技法',
 '陈防',
 '澡塘自然村',
 '莲花池',
 'hold住爱',
 'JOKER',
 '白黄']

In [1]:
mention = {'傻逼':1,'魔兽世界':2}
tokens = ['伯', '瓦尔', '·', '在', '游戏', '《', '魔兽', '世界', '》', '中', '的', '身份', '是', '？','傻逼']
mentions=[]
for t in tokens:
    for m in mention.keys():
        if m == t:
            mentions.append(t)
mentions

['傻逼']